In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# URL của trang chính
base_url = "https://elise.vn/"
res = requests.get(base_url)
soup = BeautifulSoup(res.text, 'html.parser')

# Danh sách để lưu trữ dữ liệu
data = []

In [3]:
# Hàm để lấy thông tin sản phẩm
def get_product_info(product_url, category_name, category_url,type_name):
    item = {}
    try:
        product_res = requests.get(product_url, timeout=10)  # Thêm timeout để tránh chờ lâu
        product_soup = BeautifulSoup(product_res.text, 'html.parser')
        item["category"] = category_name
        item["sub category"] = type_name
        item["product url"] = product_url

        title = product_soup.find("h1", class_="product-name").get_text().strip()
        item["title"] = title

        sku = product_soup.find("span", itemprop="sku").get_text().strip()
        item["sku"] = sku

        price_str = product_soup.find("span", class_="price").get_text().replace("\xa0", " ")
        item["price string"] = price_str

        # Lấy thông tin hình ảnh
        images_link = []
        img_list = product_soup.find("div", class_="xxx product media").find_all("img", class_="img-responsive")
        for img in img_list:
            images_link.append(img.get("data-src") or img.get("src"))

        item['images'] = images_link
    except Exception as e:
        print(f"Error fetching {product_url}: {e}")
    return item

# Lấy URL các nhóm sản phẩm
categories = soup.findAll("li", class_="category-menu level0 dropdown")

# Sử dụng ThreadPoolExecutor để truy xuất sản phẩm đồng thời
with ThreadPoolExecutor(max_workers=10) as executor:  
    futures = []
    for category in categories[:-1]:  # Bỏ mục cuối vì không phải là nhóm sản phẩm
        category_name = category.find("span").get_text()
        types = category.find("li").find_all("a")

        for type_ in types:
            type_name = type_.get_text().strip()
            category_url = type_["href"]

            # Truy cập trang nhóm sản phẩm để lấy URL các sản phẩm
            category_res = requests.get(category_url)
            category_soup = BeautifulSoup(category_res.text, 'html.parser')

            product_wrapper = category_soup.find("div", id="product-wrapper")
            if product_wrapper:
                product_items = product_wrapper.find_all("li")

                for product in product_items:
                    product_url = product.find("a")["href"]
                    futures.append(executor.submit(get_product_info, product_url, category_name, category_url,type_name))

    # Lấy kết quả từ các futures
    for future in as_completed(futures):
        result = future.result()
        if result:
            data.append(result)

In [4]:
# Tạo DataFrame từ danh sách
df = pd.DataFrame(data)

# Xuất kết quả
df

,category,sub category,product url,title,sku,price string,images
0,THỜI TRANG NỮ,Đầm,https://elise.vn/dam-2d-tho-van-nga-chan-boom-...,ĐẦM 2D THÔ VÂN NGÀ CHÂN BOOM GẤU,FS2405159DXWOWH,2.098.000 VND,[https://elise.vn/media/catalog/product/cache/...
1,THỜI TRANG NỮ,Đầm,https://elise.vn/dam-den-phoi-ren-hoa-co-duc-f...,ĐẦM ĐEN PHỐI REN HOA CỔ ĐỨC,FF2406292DIWOBK,2.298.000 VND,[https://elise.vn/media/catalog/product/cache/...
2,THỜI TRANG NỮ,Đầm,https://elise.vn/dam-ren-hoa-kem-tia-chan-dinh...,ĐẦM REN HOA KEM TỈA CHÂN ĐÍNH NƠ EO,FF2406223DILACR,2.298.000 VND,[https://elise.vn/media/catalog/product/cache/...
3,THỜI TRANG NỮ,Đầm,https://elise.vn/dam-gam-hoa-den-noi-vien-ren-...,ĐẦM GẤM HOA ĐEN NỔI VIỀN REN CỔ,FF2406287DIWOPB,2.298.000 VND,[https://elise.vn/media/catalog/product/cache/...
4,THỜI TRANG NỮ,Đầm,https://elise.vn/dam-to-den-nguc-dinh-hoa-noi-...,ĐẦM TƠ ĐEN NGỰC ĐÍNH HOA NỔI,FF2406147DIORBK,2.298.000 VND,[https://elise.vn/media/catalog/product/cache/...
...,...,...,...,...,...,...,...
356,PHỤ KIỆN,Phụ Kiện,"https://elise.vn/cai-ao-logo-da-den,-hat-trai-...","CÀI ÁO LOGO DA ĐEN, HẠT TRAI VÀ ĐÁ",FS2401177ARMENA,698.000 VND,[https://elise.vn/media/catalog/product/cache/...
357,PHỤ KIỆN,Phụ Kiện,https://elise.vn/fs2212144abpubk.html,THẮT LƯNG DA ĐEN THANH CHỮ ELISE VÀNG,FS2212144ABPUBK,598.000 VND,[https://elise.vn/media/catalog/product/cache/...
358,PHỤ KIỆN,Phụ Kiện,https://elise.vn/fs2212144abpupk.html,THẮT LƯNG DA ĐEN THANH CHỮ ELISE BẠC,FS2212144ABPUPK,598.000 VND,[https://elise.vn/media/catalog/product/cache/...
359,PHỤ KIỆN,Phụ Kiện,https://elise.vn/fs2212143abpubk.html,THẮT LƯNG DA ĐEN CHỮ ELISE VÀNG,FS2212143ABPUBK,598.000 VND,[https://elise.vn/media/catalog/product/cache/...


In [8]:
# Convert list to DataFrame
df = pd.DataFrame(data)

# Write DataFrame to CSV file
df.to_excel("C:/Users/Documents/Nhiem vu bo sung/Nhiem vu bo sung.xlsx" , index=False)